In [11]:
import numpy as np
import cv2
cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')

while True:
    ret, frame = cap.read()
    wK = cv2.waitKey(60) & 0xff
    if ret == False or wK == 27:
        break
        
    edges = cv2.Canny(frame, 100, 200)
    cv2.imshow('edges', edges)

cv2.destroyAllWindows()
cap.release()

In [24]:
import numpy as np
import cv2
cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    wK = cv2.waitKey(60) & 0xff
    if ret == False or wK == 27:
        break
    corners = cv2.goodFeaturesToTrack(gray,100,0.01,10)
    corners = np.int0(corners)
    for i in corners:
        x,y = i.ravel()
        cv2.circle(frame,(x,y),3,255,-1)
    cv2.imshow('frame',frame)

cv2.destroyAllWindows()
cap.release()

In [31]:
import numpy as np
import cv2
cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
orb = cv2.ORB_create(nfeatures=np.int(1e4))
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    wK = cv2.waitKey(60) & 0xff
    if ret == False or wK == 27:
        break
    kp, des = orb.detectAndCompute(frame, None)
    frame = cv2.drawKeypoints(frame, kp, None, color=(0,255,0), flags=0)
    cv2.imshow('frame',frame)

cv2.destroyAllWindows()
cap.release()

In [48]:
# orb brute force works
import numpy as np
import cv2
cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
orb = cv2.ORB_create(nfeatures=np.int(1e2))
#orb = cv2.ORB_create()
#cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)
ret, frame = cap.read()
scale = 1
frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
whole = 60 // scale
y, x = 250 // scale, 570 // scale
height,width,depth = frame.shape
mask = np.zeros((height,width), np.uint8)
cv2.rectangle(mask, (x,y), (x+whole,y+whole),256,thickness=-1)
masked = cv2.bitwise_and(frame, frame, mask=mask)
kp_prev, des_prev = orb.detectAndCompute(masked,None)
out = cv2.drawKeypoints(masked, kp_prev, None, color=(0,255,0), flags=0)
cv2.imshow('out', out)
cv2.waitKey(0)
cv2.destroyAllWindows()



while True:
    ret, frame = cap.read()
    
    
    if ret == True:
        frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
        height,width,depth = frame.shape
        mask = np.zeros((height,width), np.uint8)
        cv2.rectangle(mask, (x,y), (x+whole,y+whole),256,thickness=-1)
        masked = cv2.bitwise_and(frame, frame, mask=mask)
        kp, des = orb.detectAndCompute(masked,None)
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        # Match descriptors.
        matches = bf.match(des_prev,des)
        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)
        # Draw first 10 matches.
        out = cv2.drawMatches(crop,kp_prev,masked,kp,matches[:10], None, flags=2)
        cv2.imshow('out', out)

        size = len(matches)
        if size > 5:
            if size > 10:
                size = 10
            arr = np.zeros((2,size))
            for i, m in enumerate(matches[:size]):
                x, y = kp[m.trainIdx].pt
                arr[0, i] = x
                arr[1, i] = y
            mean_x, mean_y = np.int(np.mean(arr[0])), np.int(np.mean(arr[1]))
            x = mean_x - whole//2
            y = mean_y - whole//2
        cv2.rectangle(frame, (x, y), (x+whole, y+whole), (0,255,0), 1)
        cv2.imshow('frame', frame)
        crop = frame[y:y + whole, x:x + whole]
        kp_prev, des_prev = orb.detectAndCompute(crop,None)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()